<a href="https://colab.research.google.com/github/arthurvaz05/Python.Script/blob/master/Deep_Feature_Synthesis_(DFS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação Bibliotecas

In [1]:
!pip uninstall -y folium
!pip install -q folium==0.2.1
!pip uninstall -y urllib3
!pip install -q urllib3==1.25.4

Found existing installation: folium 0.2.1
Uninstalling folium-0.2.1:
  Successfully uninstalled folium-0.2.1
Found existing installation: urllib3 1.25.4
Uninstalling urllib3-1.25.4:
  Successfully uninstalled urllib3-1.25.4


In [2]:
!pip install -q --upgrade pip
!pip uninstall -y featuretools
#!pip install -q featuretools~=0.23.0
!pip install -q featuretools~=0.1.14
!pip uninstall -y pandas
!pip install -q pandas~=1.1.0
print('Bibliotecas instaladas com sucesso!!')

Found existing installation: featuretools 0.1.21
Uninstalling featuretools-0.1.21:
  Successfully uninstalled featuretools-0.1.21
Found existing installation: pandas 1.1.5
Uninstalling pandas-1.1.5:
  Successfully uninstalled pandas-1.1.5
Bibliotecas instaladas com sucesso!!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Carregar Bibliotecas

In [4]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'10aXz56ohVOgF2gH4yvmrvTJFY990RHZ1'})
data_drop.GetContentFile('dropoff_neighborhoods.csv')
data_pick = drive.CreateFile({'id':'1zakjpcMUZTKanSeVlPWbYhL85zn5g8zX'})
data_pick.GetContentFile('pickup_neighborhoods.csv')
data_trips = drive.CreateFile({'id':'1D4OJPulDK-Z5l4mgwuI2C1zBr4JRC-Q5'})
data_trips.GetContentFile('trips.pkl')
utils_file = drive.CreateFile({'id':'1UbV2z7L5vonCz3KFLywGs4U-p2g5gQwy'})
utils_file.GetContentFile('utils.py')

print('Google Drive sincronizado com sucesso!')

Google Drive sincronizado com sucesso!


In [5]:
import featuretools as ft
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils
from featuretools.primitives import (Count, Day, Hour, Max, Mean, Median, Min,
                                     Minute, Month, Std, Sum, Week, Weekday,
                                     Weekend as Weekend)
from sklearn.ensemble import GradientBoostingRegressor
from utils import (compute_features, feature_importances, load_nyc_taxi_data,
                   preview)
%matplotlib inline

print('Bibliotecas importadas com suceso!')

Bibliotecas importadas com suceso!


Exemplo 1

In [6]:
trips, pickup_neighborhoods, dropoff_neighborhoods = load_nyc_taxi_data()
preview(trips, 10)
print('Dados carregados com sucesso!')

Dados carregados com sucesso!


## Carregar dataset do Kaggle - Exemplo 2

In [9]:
!pip install kaggle

In [10]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [12]:
!kaggle datasets download "willkoehrsen/automated-feature-engineering-demo"

automated-feature-engineering-demo.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
!unzip automated-feature-engineering-demo.zip

Archive:  automated-feature-engineering-demo.zip
replace clients.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: clients.csv             
  inflating: loans.csv               
  inflating: payments.csv            


In [14]:
import pandas as pd 

basepath = "/Data Sources/automated feature engineering demo/"
clients = pd.read_csv("clients.csv")
loans = pd.read_csv("loans.csv")
payments = pd.read_csv("payments.csv")

# DFS

## Explorando os dados dos exemplos

In [15]:
trips.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,trip_duration,pickup_neighborhood,dropoff_neighborhood
0,514030,2,2016-04-02 00:00:00,2016-04-02 00:17:00,1,2.46,-73.987595,40.754711,-73.959633,40.762836,1,1039,AA,K
1,514031,1,2016-04-02 00:00:00,2016-04-02 00:24:00,2,7.90,-73.924713,40.745567,-73.972763,40.793354,1,1454,L,U
2,514032,1,2016-04-02 00:00:00,2016-04-02 00:19:00,1,1.00,-73.989800,40.732994,-74.000557,40.742622,1,1168,H,Q
3,514033,2,2016-04-02 00:00:00,2016-04-02 00:01:00,1,0.02,-73.987465,40.719822,-73.987289,40.720016,2,35,AC,AC
4,514034,1,2016-04-02 00:01:00,2016-04-02 00:58:00,2,19.00,-73.790131,40.643429,-73.975594,40.779327,1,3470,G,I


In [16]:
pickup_neighborhoods.head(5)

,neighborhood_id,latitude,longitude
0,AH,40.804349,-73.961716
1,Z,40.715828,-73.954298
2,D,40.750179,-73.992557
3,AT,40.729670,-73.981693
4,AG,40.749843,-74.003458


In [17]:
dropoff_neighborhoods.head(5)

,neighborhood_id,latitude,longitude
0,C,40.783780,-73.953145
1,S,40.836792,-73.934381
2,AL,40.818445,-73.948046
3,J,40.799573,-73.940427
4,AV,40.776270,-73.982322


In [18]:
clients.head(5)

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [19]:
loans.head(5)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13


In [20]:
payments.head(5)

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0
3,10243,2268,2002-07-20,0
4,10243,2027,2002-07-31,1


## Entidades e relações

In [21]:
# Exemplo 1 - direto, mencionando a variavel que vai ficar como index e a variavel que tem formato de data. Colocar no formato de dicionário
entities = {
    "trips": (trips, "id", 'pickup_datetime'),
    "pickup_neighborhoods": (pickup_neighborhoods, "neighborhood_id"),
    "dropoff_neighborhoods": (dropoff_neighborhoods, "neighborhood_id"),
}

In [22]:
# Exemplo 2 - podemos criar de forma mais customizada e indicando as variáveis 
#Cria o set de dataframes ou tabelas
es = ft.EntitySet(id = 'clients')
#Cria uma entity do dataset client
# Esse dataset já tem u index e time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')
#Cria uma entity do dataset loans
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')
#Cria uma entity do dataset payments
# Esse dataset não possui index mas possui time index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [23]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

## Relationships

In [24]:
# Exemplo 1 - na mesma lógica do entity, criar uma lista para indicar as relações
relationships = [("pickup_neighborhoods", "neighborhood_id", "trips", "pickup_neighborhood"),
                 ("dropoff_neighborhoods", "neighborhood_id", "trips", "dropoff_neighborhood")]

In [25]:
# Exemplo 2 - Criar diretamente no grupo de função DFS, as relações das variaveis clients e loans atraves da variavel em comum client_id
r_client_previous = ft.Relationship(es['clients']['client_id'],es['loans']['client_id'])

# Adicionar no entity set criado
es = es.add_relationship(r_client_previous)

# Exemplo 2 - Criar diretamente no grupo de função DFS, as relações das variaveis loans e payments atraves da variavel em comum loan_id
r_payments = ft.Relationship(es['loans']['loan_id'],es['payments']['loan_id'])

# Adicionar no entity set criado
es = es.add_relationship(r_payments)



## Feature Primitives

In [31]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,not,transform,"For each value of the base feature, negates the boolean value."
20,seconds,transform,Transform a Timedelta feature into the number of seconds.
21,hours,transform,Transform a Timedelta feature into the number of hours.
22,isin,transform,"For each value of the base feature, checks whether it is in a provided list."
23,days,transform,Transform a Timedelta feature into the number of days.
24,time_since,transform,Calculates time since the cutoff time.
25,mod,transform,Creates a transform feature that divides two features.
26,characters,transform,Return the characters in a given string.
27,cum_max,transform,Calculates the max of previous values of an instance for each value in a time-dependent entity.
28,multiply,transform,Creates a transform feature that multplies two features.


In [30]:
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,std,aggregation,Finds the standard deviation of a numeric feature ignoring null values.
1,count,aggregation,Counts the number of non null values.
2,median,aggregation,Finds the median value of any feature with well-ordered values.
3,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
4,num_true,aggregation,Finds the number of 'True' values in a boolean.
5,max,aggregation,Finds the maximum non-null value of a numeric feature.
6,mode,aggregation,Finds the most common element in a categorical feature.
7,sum,aggregation,Counts the number of elements of a numeric or boolean feature.
8,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
9,all,aggregation,Test if all values are 'True'.


In [32]:
#Exemplo 1 - Montar a função DSF
features1, feature_names1 = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['years', 'month', 'divide'])

print(f"Número de features: {len(feature_names1)}")
feature_names1

In [26]:
#Exemplo 2 - Além de escolher a operação Weekend importada, iremos montar o formato DFS, utilizando as listas criadas anteriores, além de ignorar variáveis
#Essa operação Weekend irá percorrer todo o dataset em busca de variáveis de tempo para criar 
trans_primitives = [Weekend]

#Criar a funçnao DFS
features2 = ft.dfs(entities =entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude", "trip_duration"]},
                  features_only=True)

print(f"Número de features: {len(features2)}")
features2

Primitivas de transformação configuradas com sucesso!


In [27]:
print(f"Número de features: {len(features2)}")
features2

Número de features: 12


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: WEEKEND(pickup_datetime)>,
 <Feature: WEEKEND(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

In [36]:
features

,income,credit_score,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),LAST(loans.rate),LAST(loans.repaid),MEAN(payments.payment_amount),MAX(payments.payment_amount),LAST(payments.payment_amount),LAST(payments.missed),MONTH(joined),income / credit_score,credit_score / income,MEAN(loans.MEAN(payments.payment_amount)),MEAN(loans.MAX(payments.payment_amount)),MEAN(loans.LAST(payments.payment_amount)),MEAN(loans.loan_amount / rate),MEAN(loans.rate / loan_amount),MAX(loans.MEAN(payments.payment_amount)),MAX(loans.LAST(payments.payment_amount)),MAX(loans.loan_amount / rate),MAX(loans.rate / loan_amount),LAST(loans.MEAN(payments.payment_amount)),LAST(loans.MAX(payments.payment_amount)),LAST(loans.MONTH(loan_start)),LAST(loans.MONTH(loan_end)),LAST(loans.loan_amount / rate),LAST(loans.rate / loan_amount),MONTH(LAST(loans.loan_start)),MONTH(LAST(loans.loan_end)),MONTH(LAST(payments.payment_date)),MAX(payments.payment_amount) / LAST(loans.rate),MEAN(loans.rate) / MAX(payments.payment_amount),MEAN(loans.loan_amount) / MAX(payments.payment_amount),MAX(loans.loan_amount) / MEAN(loans.loan_amount),MAX(loans.rate) / MEAN(loans.loan_amount),...,LAST(payments.payment_amount) / MEAN(loans.rate),MAX(loans.rate) / LAST(loans.loan_amount),LAST(payments.payment_amount) / MAX(loans.rate),MEAN(loans.loan_amount) / income,LAST(loans.loan_amount) / MEAN(loans.rate),MEAN(payments.payment_amount) / MEAN(loans.rate),credit_score / MEAN(loans.loan_amount),credit_score / MEAN(payments.payment_amount),credit_score / MAX(loans.rate),credit_score / MAX(loans.loan_amount),LAST(loans.rate) / LAST(payments.payment_amount),LAST(payments.payment_amount) / MAX(payments.payment_amount),MEAN(loans.loan_amount) / LAST(payments.payment_amount),MEAN(loans.rate) / credit_score,MAX(loans.loan_amount) / MAX(loans.rate),credit_score / MAX(payments.payment_amount),MEAN(payments.payment_amount) / credit_score,MAX(loans.loan_amount) / MEAN(payments.payment_amount),MEAN(loans.loan_amount) / MEAN(payments.payment_amount),LAST(loans.loan_amount) / income,MAX(payments.payment_amount) / MEAN(loans.loan_amount),MEAN(loans.loan_amount) / MEAN(loans.rate),MAX(loans.loan_amount) / LAST(loans.rate),LAST(payments.payment_amount) / LAST(loans.rate),MAX(payments.payment_amount) / LAST(loans.loan_amount),MAX(payments.payment_amount) / credit_score,MEAN(loans.loan_amount) / LAST(loans.rate),LAST(payments.payment_amount) / MEAN(loans.loan_amount),LAST(loans.rate) / income,MAX(loans.loan_amount) / credit_score,MEAN(loans.rate) / MAX(loans.rate),MEAN(loans.rate) / LAST(loans.loan_amount),MEAN(loans.rate) / MAX(loans.loan_amount),income / LAST(loans.rate),MAX(loans.rate) / income,LAST(loans.loan_amount) / LAST(loans.rate),LAST(loans.rate) / MAX(payments.payment_amount),MEAN(payments.payment_amount) / LAST(loans.loan_amount),MEAN(loans.loan_amount) / LAST(loans.loan_amount),income / MEAN(loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,7963.950000,3.477000,13913,9.44,home,2203,7.40,0,1178.552795,2704,241,1,10,340.454106,0.002937,1175.760996,1578.750000,1195.550000,5490.060050,0.000646,2155.142857,2402,3.650909e+04,0.003359,293.500000,368,2,8,297.702703,0.003359,2,8,11,365.405405,0.001286,2.945248,1.746997,0.001185,...,69.312626,0.004285,25.529661,0.037669,633.592177,338.956800,0.077976,0.526917,65.783898,0.044635,0.030705,0.089127,33.045436,0.005599,1473.834746,0.229660,1.897831,11.805156,6.757398,0.010420,0.339530,2290.465919,1880.135135,32.567568,1.227417,4.354267,1076.209459,0.030261,0.000035,22.404187,0.368326,0.001578,0.000250,28570.540541,0.000045,297.702703,0.002737,0.534976,3.615048,60805.867127
26326,227920,633,7270.062500,2.517500,13464,6.73,credit,5275,1.45,0,1166.736842,2658,932,1,5,360.063191,0.002777,1161.691608,1547.500000,1155.375000,5622.633622,0.000556,2143.888889,2483,2.426863e+04,0.002144,977.375000,1573,11,7,3637.931034,0.000275,11,

In [ ]:
#Exemplo 1 - Montar a função DSF automatica
features3, feature_names3 = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)